In [1]:
import pandas as pd

# Consumer Data

In [2]:
consumer_file_path = "/app/data/csv/consumer.csv"

consumer_dtypes = {"ZipCode": "str"}

consumer_df = pd.read_csv(consumer_file_path,
                 header=0,
                 index_col=0,
                 low_memory=False,
                 dtype=consumer_dtypes)

In [3]:
consumer_df.head()

,ConsumerID,EntityType,State,ZipCode,ZipCodeExtension,DateOfBirthOrFormation
0,286474,Individual,AR,72417,NaN,1978-12-17
1,16350,Individual,AR,72117,5217,1973-12-26
2,19135,Individual,AR,72019,1956,1979-09-13
3,14518,Individual,AR,71671,2527,1943-12-31
4,19762,Individual,AR,72007,9267,1943-05-28


In [4]:
consumer_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 334601 entries, 0 to 334600
Data columns (total 6 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   ConsumerID              334601 non-null  int64 
 1   EntityType              334601 non-null  object
 2   State                   326732 non-null  object
 3   ZipCode                 326731 non-null  object
 4   ZipCodeExtension        171890 non-null  object
 5   DateOfBirthOrFormation  323010 non-null  object
dtypes: int64(1), object(5)
memory usage: 17.9+ MB


# Zip Code Data

In [19]:
zip_code_file_path  = "/app/data/csv/zip_code_database.csv"

zip_code_dtypes = {"zip":"str",
                   "primary_city": "str",
                   "acceptable_cities": "str",
                   "state": "str"}

complete_zip_code_data = pd.read_csv(filepath_or_buffer=zip_code_file_path,
                                     header=0,
                                     low_memory=False,
                                     dtype=zip_code_dtypes)

In [20]:
complete_zip_code_data.head()

,zip,type,decommissioned,primary_city,acceptable_cities,unacceptable_cities,state,county,timezone,area_codes,world_region,country,latitude,longitude,irs_estimated_population
0,00501,UNIQUE,0,Holtsville,NaN,Internal Revenue Service,NY,Suffolk County,America/New_York,631,NaN,US,40.81,-73.04,562
1,00544,UNIQUE,0,Holtsville,NaN,Internal Revenue Service,NY,Suffolk County,America/New_York,631,NaN,US,40.81,-73.04,0
2,00601,STANDARD,0,Adjuntas,NaN,"Colinas Del Gigante, Jard De Adjuntas, Urb San...",PR,Adjuntas Municipio,America/Puerto_Rico,"787, 939",NaN,US,18.16,-66.72,0
3,00602,STANDARD,0,Aguada,NaN,"Alts De Aguada, Bo Guaniquilla, Comunidad Las ...",PR,Aguada Municipio,America/Puerto_Rico,"787, 939",NaN,US,18.38,-67.18,0
4,00603,STANDARD,0,Aguadilla,Ramey,"Bda Caban, Bda Esteves, Bo Borinquen, Bo Ceiba...",PR,Aguadilla Municipio,America/Puerto_Rico,"787, 939",NaN,US,18.43,-67.15,0


In [21]:
complete_zip_code_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42735 entries, 0 to 42734
Data columns (total 15 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   zip                       42735 non-null  object 
 1   type                      42735 non-null  object 
 2   decommissioned            42735 non-null  int64  
 3   primary_city              42735 non-null  object 
 4   acceptable_cities         9302 non-null   object 
 5   unacceptable_cities       11673 non-null  object 
 6   state                     42735 non-null  object 
 7   county                    41806 non-null  object 
 8   timezone                  41944 non-null  object 
 9   area_codes                41919 non-null  object 
 10  world_region              333 non-null    object 
 11  country                   42657 non-null  object 
 12  latitude                  42735 non-null  float64
 13  longitude                 42735 non-null  float64
 14  irs_es

Only care about primary_city, acceptable_cities, and state data; therefore, drop all other columns.

In [22]:
columns_to_keep = ["zip", "primary_city", "acceptable_cities", "state"]

zip_code_data = complete_zip_code_data[columns_to_keep]

#rename state column to zip_state to prevent conflict on later join
zip_code_data = zip_code_data.rename(columns={'state': 'zip_state'})

zip_code_data.head()

,zip,primary_city,acceptable_cities,zip_state
0,00501,Holtsville,NaN,NY
1,00544,Holtsville,NaN,NY
2,00601,Adjuntas,NaN,PR
3,00602,Aguada,NaN,PR
4,00603,Aguadilla,Ramey,PR


# Adding city to consumer data based on zipcode

In [23]:
consumer_df.head()

,ConsumerID,EntityType,State,ZipCode,ZipCodeExtension,DateOfBirthOrFormation
0,286474,Individual,AR,72417,NaN,1978-12-17
1,16350,Individual,AR,72117,5217,1973-12-26
2,19135,Individual,AR,72019,1956,1979-09-13
3,14518,Individual,AR,71671,2527,1943-12-31
4,19762,Individual,AR,72007,9267,1943-05-28


In [28]:
enriched_df = pd.merge(consumer_df, zip_code_data, left_on="ZipCode", right_on="zip", how="inner").drop(columns="zip", axis=1)

In [29]:
enriched_df.head()

,ConsumerID,EntityType,State,ZipCode,ZipCodeExtension,DateOfBirthOrFormation,primary_city,acceptable_cities,zip_state
0,286474,Individual,AR,72417,NaN,1978-12-17,Brookland,NaN,AR
1,241550,Individual,AR,72417,8928,1971-06-27,Brookland,NaN,AR
2,300141,Individual,AR,72417,8567,1995-08-14,Brookland,NaN,AR
3,293994,Individual,AR,72417,NaN,1992-06-12,Brookland,NaN,AR
4,82123,Individual,AR,72417,8738,1967-08-25,Brookland,NaN,AR


In [30]:
enriched_df.to_csv(path_or_buf="/app/data/csv/enriched_consumer.csv",
                   header=True
                   )

enriched_df.to_parquet(path="/app/data/parquet/enriched_consumer.parquet"
                       )